## code for reading in csv files for meds , convert meds to ATC codes

In [6]:
import os
import sys
import pandas as pd
import numpy as np
sys.path.append('../data-preprocess/')
import queryRxNorm as qrx
import json

In [65]:
col_idx_drug_indicator = 11
col_idx_med_name = 3
col_idx_reason_name = 7

In [12]:
l_all_med_files = ['./sample_data.csv']


In [13]:
## load json file for atc Hierarchy
file_hier_atc = './atcHierDict.json'
with open(file_hier_atc) as infile:
	d_hier_atc = json.load(infile)
file_atc_code_name = './atcDict.json'
with open(file_atc_code_name) as infile:
	d_atc_code_name = json.load(infile)

In [37]:
## master list of all meds
l_med_all = []
## load original data file, grab all med names
for filename in l_all_med_files:
	df_this_med_file = pd.read_csv(filename, header = None)
	### filter out just drugs
	df_this_med_file = df_this_med_file[df_this_med_file[col_idx_drug_indicator]=='D']
	## make list of all medications
	l_meds = list(df_this_med_file[col_idx_med_name])
	l_med_all = l_med_all + l_meds


In [38]:
l_med_all

['Amitriptyline',
 'Warfarin (coumadin)',
 'Cymbalta (duloxetine)',
 'Meloxicam',
 'Lexapro (citalopram)',
 'Synthroid (levothyroxine)',
 'Restasis',
 'Duragesic',
 'Alprazolam',
 'Xanax',
 'Sprintec',
 'Alprazolam Xanax',
 'Fioricet',
 'Cefdinir',
 'Zantac (ranitidine)',
 'Gabapentin',
 'Endcet',
 'Fentanyl',
 'Nexium (esomeprazole)',
 'Vesicare']

In [39]:
l_unique_med = np.unique(l_med_all)


In [60]:
d_medName_atcL2name = dict()
d_medName_atcL3name = dict()
l_medName_no_ATC = []

print "num meds: " + str(len(l_unique_med))
for idx in range(len(l_unique_med)):
	if np.mod(idx, 5) == 0:
		print idx
	med_name = l_unique_med[idx]
	s_rxcui = qrx.get_rxcui(med_name)
	l_atc_cat = qrx.rxcui_to_category(s_rxcui, relaSource="ATC")
	if len(l_atc_cat) > 0: #IF THERES A HIT
		first_code = l_atc_cat[0]
		atc_cat_L2 = d_hier_atc[first_code]["L2"]
		atc_cat_L3 = d_hier_atc[first_code]["L3"]
		name_atc_cat_L2 = d_atc_code_name[atc_cat_L2]
		name_atc_cat_L3 = d_atc_code_name[atc_cat_L3]
		# store in new dict RAW_DATAFILE_NAME => ATC NAME
		d_medName_atcL2name[med_name] = name_atc_cat_L2
		d_medName_atcL3name[med_name] = name_atc_cat_L3
	elif len(med_name.split(' ')) > 1: #try to use just the first word
		med_name_first_word = med_name.split(' ')[0]
		print med_name
		print med_name_first_word
		s_rxcui = qrx.get_rxcui(med_name_first_word)
		l_atc_cat = qrx.rxcui_to_category(s_rxcui, relaSource="ATC")
		if len(l_atc_cat) > 0: #IF THERES A HIT
			first_code = l_atc_cat[0]
			atc_cat_L2 = d_hier_atc[first_code]["L2"]
			atc_cat_L3 = d_hier_atc[first_code]["L3"]
			name_atc_cat_L2 = d_atc_code_name[atc_cat_L2]
			name_atc_cat_L3 = d_atc_code_name[atc_cat_L3]
			# store in new dict RAW_DATAFILE_NAME => ATC NAME
			d_medName_atcL2name[med_name] = name_atc_cat_L2
			d_medName_atcL3name[med_name] = name_atc_cat_L3
		else: #IF NO HIT WITH FIRST WORD
			l_medName_no_ATC.append(med_name)
	else: #IF medname is only one word  and there was no hit, then theres no hit
		l_medName_no_ATC.append(med_name)



num meds: 20
0
Alprazolam Xanax
Alprazolam
Cymbalta (duloxetine)
Cymbalta
5
10
Lexapro (citalopram)
Lexapro
Nexium (esomeprazole)
Nexium
15
Synthroid (levothyroxine)
Synthroid
Warfarin (coumadin)
Warfarin
Zantac (ranitidine)
Zantac


In [66]:
## for the datafiles, append the ATC classifications, save updated files
for filename in l_all_med_files:
	df_this_med_file = pd.read_csv(filename, header = None)
	### filter out just the drugs
	df_this_med_file = df_this_med_file[df_this_med_file[col_idx_drug_indicator]=='D']
	### append the ATC names
	l_meds = list(df_this_med_file[col_idx_med_name])
	l_atc_l2 = [d_medName_atcL2name[x] if x in d_medName_atcL2name else np.nan for x in l_meds]
	l_atc_l3 = [d_medName_atcL3name[x] if x in d_medName_atcL3name else np.nan for x in l_meds]
	df_this_med_file_to_save = df_this_med_file[[0,col_idx_reason_name, col_idx_med_name]]
	df_this_med_file_to_save['L2'] = l_atc_l2
	df_this_med_file_to_save['L3'] = l_atc_l3
	save_filename = filename.split('.csv')[0] + '_mappedATC.csv'
	df_this_med_file_to_save.to_csv(save_filename, index = None, header = None)

/Users/rchen/miniconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rchen/miniconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [61]:
d_medName_atcL2name

{'Alprazolam': u'PSYCHOLEPTICS',
 'Alprazolam Xanax': u'PSYCHOLEPTICS',
 'Amitriptyline': u'PSYCHOANALEPTICS',
 'Cefdinir': u'ANTIBACTERIALS FOR SYSTEMIC USE',
 'Cymbalta (duloxetine)': u'PSYCHOANALEPTICS',
 'Duragesic': u'ANESTHETICS',
 'Fentanyl': u'ANESTHETICS',
 'Fioricet': u'ANALGESICS',
 'Gabapentin': u'ANTIEPILEPTICS',
 'Lexapro (citalopram)': u'PSYCHOANALEPTICS',
 'Meloxicam': u'ANTIINFLAMMATORY AND ANTIRHEUMATIC PRODUCTS',
 'Nexium (esomeprazole)': u'DRUGS FOR ACID RELATED DISORDERS',
 'Restasis': u'IMMUNOSUPPRESSANTS',
 'Synthroid (levothyroxine)': u'THYROID THERAPY',
 'Vesicare': u'UROLOGICALS',
 'Warfarin (coumadin)': u'ANTITHROMBOTIC AGENTS',
 'Xanax': u'PSYCHOLEPTICS',
 'Zantac (ranitidine)': u'DRUGS FOR ACID RELATED DISORDERS'}

In [62]:
d_medName_atcL3name

{'Alprazolam': u'ANXIOLYTICS',
 'Alprazolam Xanax': u'ANXIOLYTICS',
 'Amitriptyline': u'ANTIDEPRESSANTS',
 'Cefdinir': u'OTHER BETA-LACTAM ANTIBACTERIALS',
 'Cymbalta (duloxetine)': u'ANTIDEPRESSANTS',
 'Duragesic': u'ANESTHETICS, GENERAL',
 'Fentanyl': u'ANESTHETICS, GENERAL',
 'Fioricet': u'OTHER ANALGESICS AND ANTIPYRETICS',
 'Gabapentin': u'ANTIEPILEPTICS',
 'Lexapro (citalopram)': u'ANTIDEPRESSANTS',
 'Meloxicam': u'ANTIINFLAMMATORY AND ANTIRHEUMATIC PRODUCTS, NON-STEROIDS',
 'Nexium (esomeprazole)': u'DRUGS FOR PEPTIC ULCER AND GASTRO-OESOPHAGEAL REFLUX DISEASE (GORD)',
 'Restasis': u'IMMUNOSUPPRESSANTS',
 'Synthroid (levothyroxine)': u'THYROID PREPARATIONS',
 'Vesicare': u'UROLOGICALS',
 'Warfarin (coumadin)': u'ANTITHROMBOTIC AGENTS',
 'Xanax': u'ANXIOLYTICS',
 'Zantac (ranitidine)': u'DRUGS FOR PEPTIC ULCER AND GASTRO-OESOPHAGEAL REFLUX DISEASE (GORD)'}

In [63]:
l_medName_no_ATC

['Endcet', 'Sprintec']

In [67]:
med_name

'Zantac (ranitidine)'

In [69]:
'(' in med_name and ')' in med_name

True

In [74]:
med_name.split(')')[0].split('(')[-1]

'ranitidine'